In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

2023-11-09 15:29:51.501335: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 15:29:51.501384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 15:29:51.501426: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 15:29:51.508932: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 15:29:52.327938: W tensorflow/compiler/

In [3]:
x_train, y_train, x_test, y_test = x_train[:10000], y_train[:10000], x_test[:2000], y_test[:2000]

In [4]:
x_train.shape, x_test.shape

((10000, 32, 32, 3), (2000, 32, 32, 3))

In [5]:
y_train.shape

(10000, 1)

In [6]:
import numpy as np
for label in range(10):
    print("No. {}: {}".format(label, np.sum(y_train==label)))

No. 0: 1005
No. 1: 974
No. 2: 1032
No. 3: 1016
No. 4: 999
No. 5: 937
No. 6: 1030
No. 7: 1001
No. 8: 1025
No. 9: 981


In [7]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)
y_train_one_hot.shape

(10000, 10)

In [8]:
from keras.models import Sequential
from keras.applications import VGG16
from keras.layers import Input, Flatten, Dense

In [9]:
base_model = VGG16(include_top=False, input_shape=(32,32,3), weights='imagenet')
for layer in base_model.layers:
    # block, type = layer.name.split('_')
    # if block == 'block5' and type != 'pool':
    #     layer.trainable = True
    # else:
    #     layer.trainable = False
    layer.trainable = False
    print(layer.name, layer.trainable)

2023-11-09 15:29:54.596827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-11-09 15:29:54.596861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: huy-Ubuntu
2023-11-09 15:29:54.596868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: huy-Ubuntu
2023-11-09 15:29:54.596993: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-09 15:29:54.597014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-09 15:29:54.597020: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 535.129.3 -- cannot find working devices in this configuration


input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


In [10]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=64, activation='softmax'))
model.add(Dense(units=10, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 14748170 (56.26 MB)
Trainable params: 33482 (130.79 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [11]:
model.fit(x_train, y_train_one_hot,
          validation_data=(x_test, y_test_one_hot),
          epochs=10,
          batch_size=32)

Epoch 1/10
313/313 [==============================] - 21s 64ms/step - loss: 2.2189 - accuracy: 0.2128 - val_loss: 2.1594 - val_accuracy: 0.2655
Epoch 2/10
313/313 [==============================] - 19s 60ms/step - loss: 2.0990 - accuracy: 0.3031 - val_loss: 2.0626 - val_accuracy: 0.3070
Epoch 3/10
313/313 [==============================] - 19s 60ms/step - loss: 1.9972 - accuracy: 0.3400 - val_loss: 1.9553 - val_accuracy: 0.3460
Epoch 4/10
313/313 [==============================] - 19s 60ms/step - loss: 1.8942 - accuracy: 0.3677 - val_loss: 1.8712 - val_accuracy: 0.3750
Epoch 5/10
313/313 [==============================] - 20s 64ms/step - loss: 1.8110 - accuracy: 0.4020 - val_loss: 1.8094 - val_accuracy: 0.3965
Epoch 6/10
313/313 [==============================] - 20s 64ms/step - loss: 1.7293 - accuracy: 0.4266 - val_loss: 1.7481 - val_accuracy: 0.4175
Epoch 7/10
313/313 [==============================] - 19s 60ms/step - loss: 1.6647 - accuracy: 0.4397 - val_loss: 1.7045 - val_accuracy: